# Apache Livy
https://livy.incubator.apache.org/get-started/
mac에 설치시 repl jar가 없다는 에러 남
scala 2.11을 설치해도 안됨(2.12를 지우면 되지 않을까??)
brew install scala@2.11

livy를 scala 2.12로 설치하면 실행은 되지만,
spark 실행시 livy session 에러남 (scala.tools.nsc.Settings.usejavacp 존재하지 않는 다는 에러)
scala-compiler jar파일을 확인해봤을 때, scala.tools.nsc.Settings.usejavacp 가 포함된 class파일은 존재했음

일단 EMR cluster에는 Livy가 설치되어 있으므로 패스.

## notebook
jupyter notebook에서 spark magic이라는 클라이언트 라이브러리를 통해서 spark server에 설치된 Livy를 통해 job요청을 할 수 있음.
job 진행 상황 및 spark ui 링크등을 보여줌
zeppelin의 경우 %livy.spark 등으로 진행상황등 파악 가능

## run spark on local Mac
- livy가 mac local에서 제대로 작동하지 않아. 임시로 아래 방법을 쓰면 좋을 듯
### find spark home
```
sc.getConf.get("spark.home")
```

### add env variable
```
export SPARK_HOME=/Users/hyun.kim/anaconda3/envs/spark-study2/lib/python3.9/site-packages/pyspark
export HADOOP_HOME=/Users/hyun.kim/anaconda3/envs/spark-study2/lib/python3.9/site-packages/pyspark
```

### install spark scala kernel
```
pip install spylon-kernel
python -m spylon_kernel install
```


# Livy on EMR

## change conf
`/etc/livy/conf.dist/livy.conf`

## restart livy
https://docs.cloudera.com/HDPDocuments/HDP2/HDP-2.5.0/bk_command-line-installation/content/start_stop_restart_livy.html
```
sudo -u livy sh
/usr/lib/livy/bin/livy-server stop
# after stop, it automatically start again
```

## pyFiles issue
- pyFiles를 spark-default에 추가했을 때, 인식을 안함.
- 처음에는 cluster mode 에서만 인식안되고, client mode에서는 인식됐으나, 무언가 설정 변경 후, 안됨.
- shell이나, zeppelin 에서는 정상 작동.
- livy실행할 때, pyFiles를 직접 설정해주면, 정상 작동.(우선순위 높은 conf에서 overwrite되는 문제인듯)
- 다른 spark-default의 설정은 livy에 반영되지만, pyFiles는 반영되지 않음.
- conf반영 우선순위: https://stackoverflow.com/a/47263003
    1. livy session생성할 때, post 메서드에 conf설정
    1. $LIVY_HOME/conf 의 spark-defaults.conf or livy-client.conf
    1. $SPARK_HOME/conf/spark-defaults.conf
- 아래의 코드를 bootstrap끝나고, 첫 emr step에 추가해서 해결(bootstrap에서 해도 되는지는 확인x)
    - sudo cp /etc/spark/conf.dist/spark-defaults.conf /etc/livy/conf.dist/spark-defaults.conf

In [1]:
!curl -H "Content-Type: application/json" -i http://localhost:8998/sessions

HTTP/1.1 200 OK
Date: Sat, 23 Apr 2022 13:38:53 GMT
Content-Type: application/json;charset=utf-8
Content-Length: 34
Server: Jetty(9.3.24.v20180605)

{"from":0,"total":0,"sessions":[]}

In [ ]:
import requests
import json

server_ip = 'http://localhost'
session_id = 8830
headers = {'Content-Type': 'application/json'}

In [13]:
# create session
# curl -H "Content-Type: application/json" -X POST -d '{"kind":"spark"}' -i http://localhost:8998/sessions

data = {'kind': 'spark'}
url = f'{server_ip}:8998/sessions'
response = requests.post(url, data=json.dumps(data), headers=headers)
response.json()

{'id': 8830,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ', '\nYARN Diagnostics: ']}

In [ ]:
# list livy sessions
# curl -H "Content-Type: application/json" -i http://localhost:8998/sessions

url = f'{server_ip}:8998/sessions'
response = requests.get(url, headers=headers)
response.json()

In [ ]:
# get a livy session
# curl -H "Content-Type: application/json" -i http://localhost:8998/sessions/8830


url = f'{server_ip}:8998/sessions/{session_id}'
response = requests.get(url, headers=headers)
response.json()

In [18]:
# submit job
# curl -H "Content-Type: application/json" -X POST -d '{"code":"println(sc.parallelize(1 to 5).collect())"}' -i http://localhost:8998/sessions/0/statements

data = {'code': 'println(sc.parallelize(1 to 5).collect())'}
url = f'{server_ip}:8998/sessions/{session_id}/statements'
response = requests.post(url, data=json.dumps(data), headers=headers)
response.json()

{'id': 0,
 'code': 'println(sc.parallelize(1 to 5).collect())',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [19]:
# get job output
# curl -H "Content-Type: application/json" -i http://localhost:8998/sessions/0/statements/0
url = f'{server_ip}:8998/sessions/{session_id}/statements/0'
response = requests.get(url, headers=headers)
response.json()

{'id': 0,
 'code': 'println(sc.parallelize(1 to 5).collect())',
 'state': 'available',
 'output': {'status': 'ok',
  'execution_count': 0,
  'data': {'text/plain': '[I@12a0d40b\n'}},
 'progress': 1.0,
 'started': 1650722071248,
 'completed': 1650722076537}

In [22]:
# delete session
# curl -H "Content-Type: application/json" -X DELETE -d -i http://localhost:8998/sessions/0
url = f'{server_ip}:8998/sessions/{session_id}'
response = requests.delete(url, headers=headers)
response.json()


{'msg': 'deleted'}